In [3]:
import audio_metadata
import datetime as dt
from tinytag import TinyTag
import os
import pandas as pd

In [232]:
# examine items in Music directory
ls '/path/Music/iTunes/iTunes Media/Music'

Art Blakey & The Jazz Messengers/ Miles Davis/
Bell X1/                          Oscar Peterson/
Bonobo/                           Phoenix/
Compilations/                     Radiohead/
Daft Punk/                        Real Estate/
George Fitzgerald/                Röyksopp/
Hungry Kids of Hungary/           The 1975/
John Coltrane/                    The Avalanches/
José González/                    The Dodos/
Justin Bieber/                    The Mighty Bop/
LCD Soundsystem/                  The Paper Kites/
Lapalux/                          Thelonious Monk/
Laszlo Dancehall/                 Toro y Moi/
Leon Vynehall/                    Unknown Artist/
Local Natives/                    Washed Out/
McCoy Tyner/


In [16]:
# path to directory
path = '/path/Music/iTunes/iTunes Media/Music/'

In [17]:
# initialize dataframe
df_music = pd.DataFrame()

In [18]:
# define metadata categories to become table columns
artists = []
albums = []
songs = []
duration = []
year_released = []
genre = []
file_size = []
bitrate = []

# define file extentions
extentions = ('.m4a', '.mp3')

# iterate through all folders in Music (this represents artists in the local music library)
for i in os.listdir(path):
    
    # iterate through all folders in artists (this represents albums in the local music library)
    for j in os.listdir(f'{path}{i}'):
        
        # if j is a directory
        if os.path.isdir(f'{path}{i}/{j}'):
            
            # iterate through songs in albums
            for k in os.listdir(f'{path}{i}/{j}'):
                
                # if the song is a .m4a or .mp3 file,
                if k.endswith(extentions):
                    
                    # get metadata of each file in the directory and assign to song
                    song = TinyTag.get(f'{path}{i}/{j}/{k}')
                    
                    # get song titles
                    songs += [song.title]
                    # get duration
                        # convert TinyTag duration tag from float to int with round
                        # convert each int to string formatted time with dt.datetime.utcfromtimestamp
                    duration += [dt.datetime.strftime(dt.datetime.utcfromtimestamp(round(song.duration)), "%M:%S")]
                    # get artists  
                    artists += [song.artist]
                    # get albums
                    albums += [song.album]
                    # get year released (firts four characters of string)
                    year_released += [str(song.year)[:4]]
                    # get genre
                    genre += [song.genre]
                    # get file size
                    file_size += [str(round(song.filesize/1000000, 2)) + " MB"]
                    # get bitrate
                    bitrate += [str(round(song.bitrate)) + ' kbps']

In [19]:
# add lists to dataframe
df_music['Song'] = songs
df_music['Duration'] = duration
df_music['Artist'] = artists
df_music['Album'] = albums
df_music['YearReleased'] = year_released
df_music['Genre'] = genre
df_music['FileSize'] = file_size
df_music['Bitrate'] = bitrate

In [20]:
# examine dataframe
df_music.head()

,Song,Duration,Artist,Album,YearReleased,Genre,FileSize,Bitrate
0,What Do You Mean?,03:26,Justin Bieber,Purpose,2015,Pop,7.26 MB,256 kbps
1,But Not for Me,09:42,John Coltrane,My Favorite Things,1961,Jazz,21.69 MB,256 kbps
2,Summertime,11:38,John Coltrane,My Favorite Things,1961,Jazz,24.73 MB,256 kbps
3,My Favorite Things,13:44,John Coltrane,My Favorite Things,1961,Jazz,32.07 MB,256 kbps
4,Everytime We Say Goodbye,05:45,John Coltrane,My Favorite Things,1961,Jazz,13.36 MB,256 kbps


In [4]:
# SQL connection
from sqlalchemy import create_engine
import pymysql

# estabish connection
engine = create_engine('mysql+pymysql://root:!@#Mu$iqu3!?@localhost')

In [2]:
# create music database
engine.execute("CREATE DATABASE music;")

In [30]:
# create songs table
engine.execute('CREATE TABLE songs (\
    SongID INT NOT NULL AUTO_INCREMENT,\
    Song VARCHAR(100),\
    Duration TIME,\
    Artist VARCHAR(100),\
    Album VARCHAR(100),\
    YearReleased VARCHAR(4),\
    Genre VARCHAR(50),\
    FileSize VARCHAR(50),\
    Bitrate VARCHAR(50),\
    PRIMARY KEY(SongID)\
    ) AUTO_INCREMENT = 1;')

In [31]:
# write to songs table
df_music.to_sql('songs', con=engine, if_exists='append', index=False)

In [3]:
# dispose connection
engine.dispose()